In [25]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from src.features.conferences import ConferenceFeatures
from src.features.coaches import CoachFeatures
from src.utils import load_data_template


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
data = load_data_template(season=False)
data.dropna(subset=['a_win'], inplace=True)
coach_feat = CoachFeatures(default_lags=0)
data = coach_feat.per_team_wrapper(data, coach_feat.coach_func, per_game=False, per_day=False)
# data = coach_feat.per_team_wrapper(data, coach_feat.coach_years, per_game=False, per_day=False)
data.dropna(inplace=True)
print(data.shape)
data.sample()

(2146, 9)


,Season,team_a,team_b,in_target,game_set,a_win,DayNum,coach_team_a,coach_team_b
9759,1995,1207,1462,False,0,True,137,712,1201


In [28]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss


X = data.drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target', 'DayNum'], axis=1)
y = data[['a_win']].astype(int)

kf = KFold(n_splits=5, shuffle=True)
metrics = []
for tr_i, t_i in kf.split(X):
    X_tr, y_tr = X.iloc[tr_i], y.iloc[tr_i]
    X_t, y_t = X.iloc[t_i], y.iloc[t_i]
    xgb = XGBClassifier()
    xgb.fit(X_tr.values, y_tr.values.reshape(-1))
    preds = xgb.predict_proba(X_t.values)
    metric = log_loss(y_t.values, preds)
    print(metric)
    metrics.append(metric)
    
print('Metric Mean: {:.2f} ({:.2f})'.format(np.mean(metrics), np.std(metrics)))

0.672972887304
0.681665985526
0.680357980353
0.680069103073
0.692334373757
Metric Mean: 0.68 (0.01)
